# Bibliotecas e Configurações

1. Requer:
    * `./data/GlobalLandTemperaturesByCity.csv` e resulta em `./out/parsed/tpr/~.csv`;
    * `./data/co2_data.csv` (gerado com o `convert.ipynb`) e resulta em `./out/parsed/co2/~.csv`.

<br />

2. É necessário possuir o **Java SE Development Kit 11** (JDK) instalado;
    * Outras versões apresentam problemas;
    * Possivel baixar o `jdk-11.0.16` através desse *[link](https://gist.github.com/wavezhang/ba8425f24a968ec9b2a8619d7c2d86a6)*;
    * Apenas dar `Crtl+F` em `jdk-11.0.16`, o site oficial requer *login*, então acho essa opção melhor.

<br />

3. Além disso, é necessário definir a variável de ambiente `JAVA_HOME` cujo valor é o diretório de instalação do JDK;
    * Exemplo: `C:\Progra~1\Java\jdk-11.0.16`;
    * `Progra~1` é, na verdade, a pasta `Program Files`, mas escrita sem espaços, para evitar *bugs*.

<br />

4. Definir também as variáveis de ambiente:
    * `PYSPARK_DRIVER_PYTHON` = `jupyter`;
    * `PYSPARK_PYTHON` = `python`.

<br />

5. Além disso tudo para salvar arquivos é necessário o Spark + Hadoop;
    1. Assim sendo, efetue o download da versão 3.2.2 do [Spark](https://spark.apache.org/downloads.html);
    2. Extraia os arquivos para um diretório desejado do computador (local de instalação);
    3. Dentro desse diretório de instalação do Spark crie o diretório do Haddop: `<spark_path>\hadoop\bin`;
    4. Efetuar o download da versão 3.3.1 do [Hadoop](https://github.com/kontext-tech/winutils)
        * Aqui existe uma gambiarra:
            1. Baixar o repositório como ZIP;
            2. Extrair todos os arquivos da respectiva versão na pasta `bin` recém criada;
            3. Dentro dessa pasta `bin`, criar uma nova pasta `bin` e repetir o passo anterior.

<br />

6. Definir as variáveis de ambiente:
    * `SPARK_HOME`: diretório de instalação do Spark;
    * `HADOOP_HOME` = `%SPARK_HOME%\hadoop\bin`.

<br />

7. Adicionar os caminhos dessa variáveis (`SPARK_HOME` E `HADOOP_HOME`) ao Path;

<br />

8. Será necessário reiniciar o computador após definir as variáveis de ambiente e alterar o Path.
    * Há abaixo uma verificação para atestar se é possível ler todas as variáveis necessárias.

In [1]:
import os

environ_vars = [
    'JAVA_HOME',
    'PYSPARK_DRIVER_PYTHON',
    'PYSPARK_PYTHON',
    'SPARK_HOME',
    'HADOOP_HOME'
]

for var in environ_vars:
    if not os.environ.get(var):
        print(f'AVISO: Variável {var} não encontrada.')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Leitura

In [3]:
spark = SparkSession.builder.getOrCreate()

tpr_data = spark.read.options(header='True').csv('./data/GlobalLandTemperaturesByCity.csv')
co2_data = spark.read.options(header='True').csv('./data/co2_data.csv')

In [4]:
tpr_data.show(5)

print('Número de registros:', tpr_data.count())

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows

Número de registros: 8599212


*Dataset* de temperaturas:

* `dt` (string): mês referente à amostra;
* `AverageTemperature` (float): temperatura média global;
* `AverageTemperatureUncertainty` (float): incerteza associada à temperatura medida;
* `City` (string): cidade de origem da amostra;
* `Country` (string): país de origem da amostra;
* `Latitude` (string): coordenadas de latitude em graus da amostra, com valor e ponto cardeal;
* `Longitude` (string): coordenadas de longitude em graus da amostra, com valor e ponto cardeal;

In [5]:
co2_data.show(5)

print('Número de registros:', co2_data.count())

+------+------+----------+---------+--------+------------------+
|LonDim|LatDim|     Times|Longitude|Latitude|             value|
+------+------+----------+---------+--------+------------------+
|     0|     0|1850-01-01|   -179.5|    89.5| 288.1340637207031|
|     0|     0|1850-02-01|   -179.5|    89.5|288.52105712890625|
|     0|     0|1850-03-01|   -179.5|    89.5| 288.8883972167969|
|     0|     0|1850-04-01|   -179.5|    89.5| 289.0948486328125|
|     0|     0|1850-05-01|   -179.5|    89.5| 289.0404052734375|
+------+------+----------+---------+--------+------------------+
only showing top 5 rows

Número de registros: 127526400


*Dataset* de temperaturas:

* `LatDim` (int): ?
* `LonDim` (int): ?
* `Times` (string): mês referente à amostra;
* `Longitude` (float): graus de longitude com referência no Meridiano de Greenwich;
* `Latitude` (float): graus de latitude com referência na Linha do Equador;
* `value` (float): concentração de $CO_2$ em ppm;

# Tratamento

Separado entre temperaturas e gás carbônico.

## Temperatura

* Define uma mesma data de início para ambos os datasets

In [6]:
tpr_data = tpr_data[tpr_data['dt'] >= '1850-01-01']

* Padroniza nomes de colunas entre os datasets (prefixa aquelas em comum)

In [7]:
columns_rename = [
    ('dt', 't_date'),
    ('AverageTemperature', 'tpr'),
    ('AverageTemperatureUncertainty', 'tpr_unc'),
    ('City', 'city'),
    ('Country', 'country'),
    ('Longitude', 't_lon'),
    ('Latitude', 't_lat'),
]

for old, new in columns_rename:
    tpr_data = tpr_data.withColumnRenamed(old, new)

* Trata coordenadas:
    1. Extrai ponto cardeal para um coluna prórpia;
    2. Remove ponto cardeal do valor da coordenada e a converte para `float`.
    3. Aplica a lógica de valores negativos e positivos consoante ponto cardeal
    4. Remove colunas desncessárias

In [8]:
# Trata coordenadas

geoposition_cols = ['t_lon', 't_lat']

for c in geoposition_cols:
    tpr_data = tpr_data.withColumn(
        c + '_cardinal',
        expr(f"RIGHT({c}, 1)")
    )

    tpr_data = tpr_data.withColumn(
        c,
        expr(f"FLOAT( LEFT({c}, LENGTH({c})-1) )")
    )

In [9]:
tpr_data = tpr_data.withColumn(
    't_lon',
    expr(
        """
        CASE
            WHEN t_lon_cardinal = 'E' THEN  t_lon
            WHEN t_lon_cardinal = 'W' THEN -t_lon
            ELSE NULL
        END
        """
    )
)

tpr_data = tpr_data.withColumn(
    't_lat',
    expr(
        """
        CASE
            WHEN t_lat_cardinal = 'N' THEN  t_lat
            WHEN t_lat_cardinal = 'S' THEN -t_lat
            ELSE NULL
        END
        """
    )
)

In [10]:
cols_to_drop = ['t_lon_cardinal', 't_lat_cardinal']

for c in cols_to_drop:
    tpr_data = tpr_data.drop(c)

## Gás Carbônico

* Padroniza nomes de colunas entre os datasets (prefixa aquelas em comum)
* Remove colunas não utilizadas

In [11]:
columns_rename = [
    ('Times', 'c_date'),
    ('Longitude', 'c_lon'),
    ('Latitude', 'c_lat'),
    ('value', 'co2'),
]

for old, new in columns_rename:
    co2_data = co2_data.withColumnRenamed(old, new)

In [12]:
cols_to_drop = ['LonDim', 'LatDim']

for c in cols_to_drop:
    co2_data = co2_data.drop(c)

# Exportação

In [13]:
tpr_data.show(5)

+----------+--------------------+-------+-----+-------+-----+-----+
|    t_date|                 tpr|tpr_unc| city|country|t_lat|t_lon|
+----------+--------------------+-------+-----+-------+-----+-----+
|1850-01-01|              -5.265|   1.82|Århus|Denmark|57.05|10.33|
|1850-02-01|               1.859|  1.641|Århus|Denmark|57.05|10.33|
|1850-03-01|0.031999999999999806|  3.167|Århus|Denmark|57.05|10.33|
|1850-04-01|  5.7639999999999985|  1.903|Århus|Denmark|57.05|10.33|
|1850-05-01|              11.037|  0.586|Århus|Denmark|57.05|10.33|
+----------+--------------------+-------+-----+-------+-----+-----+
only showing top 5 rows



In [14]:
co2_data.show(5)

+----------+------+-----+------------------+
|    c_date| c_lon|c_lat|               co2|
+----------+------+-----+------------------+
|1850-01-01|-179.5| 89.5| 288.1340637207031|
|1850-02-01|-179.5| 89.5|288.52105712890625|
|1850-03-01|-179.5| 89.5| 288.8883972167969|
|1850-04-01|-179.5| 89.5| 289.0948486328125|
|1850-05-01|-179.5| 89.5| 289.0404052734375|
+----------+------+-----+------------------+
only showing top 5 rows



Escrita dos datasets parseados, descomentar se necessário. 

* **Temperatura:** aproximadamente 30 segundos de execução e 400 MB de saída;
* **Gás carbônico:** aproximadamente 6 minutos de execução e 5 GB de saída;

In [15]:
# tpr_data.coalesce(1).write.format('csv').option('header', True).mode('overwrite').save('./out/parsed/tpr')

In [16]:
# co2_data.coalesce(1).write.format('csv').option('header', True).mode('overwrite').save('./out/parsed/co2')